## Importing libraries

In [44]:
import os
import pandas as pd
import zipfile
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from rapidfuzz import fuzz, process
import folium
from scipy import stats
import fiona
from typing import Union
import polyline  
from shapely.geometry import shape, mapping, Point, LineString, MultiLineString, Polygon
from shapely.ops import linemerge, unary_union, transform
from pyproj import Transformer
import json
import requests


# Functions to load the data

In [2]:
data_folder = "data"

# 1. Load bus routes data from CSV file
def load_bus_routes_data():
    bus_routes_path = os.path.join(data_folder, "bus_routes_data.csv")
    if os.path.exists(bus_routes_path):
        bus_routes_df = pd.read_csv(bus_routes_path)
        return bus_routes_df
    else:
        print(f"Error: {bus_routes_path} not found.")
        return None

# 2. Load bus stops data from CSV file and convert to GeoDataFrame
def load_bus_stops_data():
    bus_stops_path = os.path.join(data_folder, "bus_stops_data.csv")
    if os.path.exists(bus_stops_path):
        bus_stops_df = pd.read_csv(bus_stops_path)
        # Create a GeoDataFrame for bus stops with Point geometries from Longitude and Latitude
        bus_stops_gdf = gpd.GeoDataFrame(
            bus_stops_df,
            geometry=gpd.points_from_xy(bus_stops_df['Longitude'], bus_stops_df['Latitude']),
            crs='EPSG:4326'
        )
        print("Bus Stops Data Loaded and converted to GeoDataFrame.")
        return bus_stops_gdf
    else:
        print(f"Error: {bus_stops_path} not found.")
        return None

# 3. Load Passenger Volume by Bus Stops data from ZIP file
def load_passenger_volume_bus_stops():
    zip_path = os.path.join(data_folder, "transport_node_bus_202408.zip")
    csv_file_name = "transport_node_bus_202408.csv"
    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as z:
            with z.open(csv_file_name) as csv_file:
                passenger_volume_df = pd.read_csv(csv_file)
                print("Passenger Volume by Bus Stops Data Loaded.")
                return passenger_volume_df
    else:
        print(f"Error: {zip_path} not found.")
        return None
    
# 4. Load Origin-Destination Bus Stops data from ZIP file
# def load_od_volume_bus_stops():
#     zip_path = os.path.join(data_folder, "origin_destination_bus_202408.zip")
#     csv_file_name = "origin_destination_bus_202408.csv"
#     if os.path.exists(zip_path):
#         with zipfile.ZipFile(zip_path, 'r') as z:
#             with z.open(csv_file_name) as csv_file:
#                 od_volume_df = pd.read_csv(csv_file)
#                 print("Origin-Destination Bus Stops Data Loaded.")
#                 return od_volume_df
#     else:
#         print(f"Error: {zip_path} not found.")
#         return None

def load_od_volume_bus_stops(date):
    filename = "origin_destination_bus_" + date + ".zip"
    zip_path = os.path.join(data_folder, filename)
    csv_file_name = "origin_destination_bus_" + date + ".csv"
    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as z:
            with z.open(csv_file_name) as csv_file:
                od_volume_df = pd.read_csv(csv_file)
                print(f"Origin-Destination Bus Stops for {date} Data Loaded.")
                return od_volume_df
    else:
        print(f"Error: {zip_path} not found.")
        return None
    
# 5. Load MRT exits data from ZIP file
def load_mrt_exits_shapefile():
    zip_path = os.path.join('data', "train_station_exit_geospatial_whole_island_202408.zip")
    shapefile_components = [
        'TrainStationExit/Train_Station_Exit_Layer.shp',
        'TrainStationExit/Train_Station_Exit_Layer.dbf',
        'TrainStationExit/Train_Station_Exit_Layer.shx'
    ]

    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        
            temp_dir = 'temp_shapefile'  # Temporary folder for extraction
            os.makedirs(temp_dir, exist_ok=True)
            
            # Extract necessary shapefile components
            for component in shapefile_components:
                zip_ref.extract(component, path=temp_dir)

            # Load the shapefile into a GeoDataFrame
            shapefile_path = os.path.join(temp_dir, 'TrainStationExit', 'Train_Station_Exit_Layer.shp')
            gdf_exits = gpd.read_file(shapefile_path)

            # Clean up extracted files and temporary directory
            for component in shapefile_components:
                os.remove(os.path.join(temp_dir, component))
            
            # Remove the empty directory
            try:
                os.rmdir(os.path.join(temp_dir, 'TrainStationExit'))
                os.rmdir(temp_dir)
            except OSError as e:
                print(f"Warning: {e.strerror} - {e.filename}")
            
            print("MRT Exits data loaded.")
            
            # Display the columns (fields) and first few rows in the shapefile
            print("Columns in MRT Exits Shapefile:")
            print(gdf_exits.columns)
            print("\nFirst 5 rows of data:")
            print(gdf_exits.head())
            
            return gdf_exits
    else:
        print(f"Error: {zip_path} not found.")
        return None
    
# 6. Load mrtlines data
def load_mrt_lines_mapping():
    mrt_lines_path = os.path.join(data_folder, "singapore_mrt_stations_with_lines_filtered.csv")
    if os.path.exists(mrt_lines_path):
        mrt_lines_df = pd.read_csv(mrt_lines_path)
        return mrt_lines_df
    else:
        print(f"Error: {mrt_lines_path} not found.")

# 7. Load LTA Datamall Geospatial data 
def load_mrt_shapefile():
    zip_path = os.path.join('data', "train_station_geospatial_whole_island_202408.zip")
    shapefile_components = [
        'TrainStation_Jul2024/RapidTransitSystemStation.shp',
        'TrainStation_Jul2024/RapidTransitSystemStation.shx',
        'TrainStation_Jul2024/RapidTransitSystemStation.dbf',
        'TrainStation_Jul2024/RapidTransitSystemStation.prj'
    ]

    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        
            temp_dir = 'temp_shapefile'  # Temporary folder for extraction
            os.makedirs(temp_dir, exist_ok=True)
            
            # Extract necessary shapefile components
            for component in shapefile_components:
                zip_ref.extract(component, path=temp_dir)

            shapefile_path = os.path.join(temp_dir, 'TrainStation_Jul2024', 'RapidTransitSystemStation.shp')
            features = []    
            with fiona.open(shapefile_path, 'r') as src:
                for feature in src:
                    try:
                        geom = shape(feature['geometry'])  # Create a valid geometry
                        properties = feature['properties']
                        properties['geometry'] = geom
                        features.append(properties)
                    except Exception as e:
                        print(f"Skipped feature due to error: {e}")
            # Load the shapefile into a GeoDataFrame
            gdf_mrt = gpd.GeoDataFrame(features)

            # Clean up extracted files and temporary directory
            for component in shapefile_components:
                os.remove(os.path.join(temp_dir, component))
            
            # Remove the empty directory
            try:
                os.rmdir(os.path.join(temp_dir, 'TrainStationExit'))
                os.rmdir(temp_dir)
            except OSError as e:
                print(f"Warning: {e.strerror} - {e.filename}")
            
            print("MRT Exits data loaded.")
            
            # Display the columns (fields) and first few rows in the shapefile
            print("Columns in MRT Exits Shapefile:")
            print(gdf_mrt.columns)
            print("\nFirst 5 rows of data:")
            print(gdf_mrt.head())
            
            return gdf_mrt
    else:
        print(f"Error: {zip_path} not found.")
        return None
    
# 8. Load Bus Services Data from CSV file
def load_bus_services():
    bus_services_path = os.path.join(data_folder, "BusServicesInfo.csv")
    if os.path.exists(bus_services_path):
        all_buses_from_lta = pd.read_csv(bus_services_path)
        print("Bus Services data loaded successfully.")
        return all_buses_from_lta
    else:
        print(f"Error: {bus_services_path} not found.")
        return None
    
# 9. Load Planning Area Names Data from CSV file
def load_planning_area_names():
    planning_area_path = os.path.join(data_folder, "PlanningAreaNames.csv")
    if os.path.exists(planning_area_path):
        df_planning_area = pd.read_csv(planning_area_path)
        print("Planning area names data loaded successfully.")
        return df_planning_area
    else:
        print(f"Error: {planning_area_path} not found.")
        return None
    
# 10. Load Population Data from CSV file
def load_population_data():
    population_data_path = os.path.join("data", "population_planning_area_data.csv")
    if os.path.exists(population_data_path):
        df_population_planning_area = pd.read_csv(population_data_path)
        print("Population data loaded successfully.")
        return df_population_planning_area
    else:
        print(f"Error: {population_data_path} not found.")
        return None


# 11. Load Planning Area Data from CSV file
def load_planning_area_data():
    planning_area_path = os.path.join(data_folder, "planning_area_geojson_data.csv")
    if os.path.exists(planning_area_path):
        planning_area_location = pd.read_csv(planning_area_path)
        print("Planning area data loaded successfully.")
        return planning_area_location
    else:
        print(f"Error: {planning_area_path} not found.")
        return None

# # 2. Load Planning Area Data from CSV file
# def load_planning_area_data():
#     planning_area_path = os.path.join(data_folder, "planning_area_geojson_data.csv")
#     if os.path.exists(planning_area_path):
#         planning_area_location = pd.read_csv(planning_area_path)
#         print("Planning area data loaded successfully.")
#         return planning_area_location
#     else:
#         print(f"Error: {planning_area_path} not found.")
#         return None

In [3]:
# Load datasets
bus_routes_df = load_bus_routes_data()
bus_stops_gdf = load_bus_stops_data()
passenger_volume_df = load_passenger_volume_bus_stops()
jul24_od_volume_df = load_od_volume_bus_stops("202407")
aug24_od_volume_df = load_od_volume_bus_stops("202408")
sep24_od_volume_df = load_od_volume_bus_stops("202409")
mrt_exits_gdf = load_mrt_exits_shapefile()
mrt_lines_mapping = load_mrt_lines_mapping()
mrt_gdf = load_mrt_shapefile()
all_buses_from_lta = load_bus_services()
bus_services_info_df = pd.read_csv("data/BusServicesInfo.csv")
df_planning_area = load_planning_area_names()
df_population_planning_area = load_population_data()
planning_area_location = load_planning_area_data()
planning_area_location = load_planning_area_data()

# Ensure that the data is being loaded correctly
if bus_routes_df is not None:
    print(f"Bus Routes DataFrame shape: {bus_routes_df.shape}")
if bus_stops_gdf is not None:
    print(f"Bus Stops GeoDataFrame shape: {bus_stops_gdf.shape}")
if passenger_volume_df is not None:
    print(f"Passenger Volume DataFrame shape: {passenger_volume_df.shape}")
if jul24_od_volume_df is not None:
    print(f"Jul24 Passenger Volume by Origin-Destination Bus Stops DataFrame shape: {jul24_od_volume_df.shape}")
if aug24_od_volume_df is not None:
    print(f"Aug24 Passenger Volume by Origin-Destination Bus Stops DataFrame shape: {aug24_od_volume_df.shape}")
if sep24_od_volume_df is not None:
    print(f"Sep24 Passenger Volume by Origin-Destination Bus Stops DataFrame shape: {sep24_od_volume_df.shape}")
if mrt_exits_gdf is not None:
    print(f"MRT Exits GeoDataFrame shape: {mrt_exits_gdf.shape}")
if mrt_lines_mapping is not None:
    print(f"MRT Lines Mapping DataFrame shape: {mrt_lines_mapping.shape}")
if mrt_gdf is not None:
    print(f"MRT GeoDataFrame shape: {mrt_gdf.shape}")
    

Bus Stops Data Loaded and converted to GeoDataFrame.
Passenger Volume by Bus Stops Data Loaded.
Origin-Destination Bus Stops for 202407 Data Loaded.
Origin-Destination Bus Stops for 202408 Data Loaded.
Origin-Destination Bus Stops for 202409 Data Loaded.
MRT Exits data loaded.
Columns in MRT Exits Shapefile:
Index(['stn_name', 'exit_code', 'geometry'], dtype='object')

First 5 rows of data:
                 stn_name exit_code                     geometry
0  MACPHERSON MRT STATION    Exit A  POINT (34285.068 34322.985)
1  MACPHERSON MRT STATION    Exit B  POINT (34382.153 34231.904)
2  MACPHERSON MRT STATION    Exit C  POINT (34337.292 34190.603)
3    TONGKANG LRT STATION    Exit B  POINT (33872.145 41256.053)
4    TONGKANG LRT STATION    Exit A  POINT (33858.542 41234.065)
MRT Exits data loaded.
Columns in MRT Exits Shapefile:
Index(['ATTACHEMEN', 'STN_NAM', 'STN_NAM_DE', 'TYP_CD', 'TYP_CD_DES',
       'geometry'],
      dtype='object')

First 5 rows of data:
  ATTACHEMEN STN_NAM      

C:\Users\brand\AppData\Local\Temp\ipykernel_12116\2851669825.py:155: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  properties['geometry'] = geom


<h1>Plotting all bus routes available in Singapore using the OSRM API</h1>

We first grouped the bus routes by 'ServiceNo' and 'Direction'. To ensure that the route follows the correct order of bus stops, we then sorted the data by 'StopSequence'.
For each consecutive pair of bus stops, we made an OSRM query to fetch the encoded polyline between them. The encoded polyline shows the route between the two stops via road network. 
The encoded polylines are then stored into a Dataframe and saved to a csv file for further analysis.
To represent the bus routes, we then decoded the encoded polylines and displayed them on a Folium map.

Note: Run the first code chunk to test with one route first. 

In [46]:
bus_df_w_routes_and_coordinates = pd.merge(bus_routes_df, bus_stops_gdf, on='BusStopCode', how='left')

In [ ]:
def get_osrm_encoded_route(start_coords, end_coords):
    osrm_url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}?geometries=polyline"
    response = requests.get(osrm_url)
    if response.status_code == 200:
        data = response.json()
        if 'routes' in data and len(data['routes']) > 0:
            return data['routes'][0]['geometry']  ## returns encoded polyline
    return None

## creating sg map
map_of_all_bus_routes = folium.Map(location=[1.3521, 103.8198], zoom_start=12)
unique_services = bus_df_w_routes_and_coordinates['ServiceNo'].unique()[:600]

## filter for unique buses
filtered_bus_df = bus_df_w_routes_and_coordinates[bus_df_w_routes_and_coordinates['ServiceNo'].isin(unique_services)]
bus_routes_grouped = filtered_bus_df.groupby(['ServiceNo', 'Direction'])


output_data = []
line_color = 'blue'  


for (service_no, direction), group in bus_routes_grouped:
    
    group = group.sort_values(by='StopSequence') ## sort group by stopsequence 
    bus_stops = list(zip(group['Latitude'], group['Longitude']))
    
    ## get subsequent roads for next busstops
    for i in range(len(bus_stops) - 1):
        start_coords = bus_stops[i]
        end_coords = bus_stops[i + 1]
        
        
        encoded_route = get_osrm_encoded_route(start_coords, end_coords) ##encoded route between busstops
        if encoded_route:
            output_data.append({
                "ServiceNo": service_no,
                "Direction": direction,
                "EncodedPolyline": encoded_route
            })
            
            decoded_route = polyline.decode(encoded_route) ## decode encoded polyline to lat long
            
            ## map decoded route on the sg map
            folium.PolyLine(decoded_route, color=line_color, weight=2.5, opacity=1,
                            popup=f"Bus {service_no} Direction {direction}").add_to(map_of_all_bus_routes)

## convert output data to df
df_encoded_polylines = pd.DataFrame(output_data)
## save to csv
df_encoded_polylines.to_csv('data/encoded_polylines_output.csv', index=False)
map_of_all_bus_routes


In [4]:
df_encoded_polylines = pd.read_csv(r"data/encoded_polylines_output.csv")
df_encoded_polylines

,ServiceNo,Direction,EncodedPolyline
0,10,1,ekgGemlyRfAhAeB~Au@L?vDxE@
1,10,1,_ggGsalyRhJAvCKjImA
2,10,1,qlfGodlyRrEu@xCeAbj@sb@
3,10,1,_vdG_lmyRdGqEtGsF
4,10,1,cedGezmyR`McKnE~M
...,...,...,...
24773,9B,1,oknG}_wyRp@aBr@_A|AiApGmD
24774,9B,1,y|mGwlwyRhDkBVnANLV@dB_@j@J
24775,9B,1,}pmGwmwyR`WnI
24776,9B,1,{xlGgcwyRbW`I


## Aggregating all the geometries from one bus stop to another bus stop into a Singular Geometry object by Bus Service Number 

In [37]:
# Step 1: Function to decode polyline and convert to LineString
def decode_polyline_to_geometry(encoded_polyline):
    # Decode polyline into a list of (lat, lon) tuples
    coords = polyline.decode(encoded_polyline)
    # Convert to LineString object (lon, lat for shapely)
    return LineString([(lon, lat) for lat, lon in coords])

# Step 2: Apply the decoding function to the 'EncodedPolyLine' column
df_encoded_polylines['geometry'] = df_encoded_polylines['EncodedPolyline'].apply(decode_polyline_to_geometry)

# Step 3: Group the dataframe by ServiceNo and collect all LineStrings into a list
df_bus_grouped_geometry = df_encoded_polylines.groupby('ServiceNo')['geometry'].apply(list)
# Use linemerge to create a single MultiLineString geometry object 
df_bus_grouped_geometry = df_bus_grouped_geometry.apply(lambda x: linemerge(MultiLineString(x)))

# Step 3: Convert the grouped data back into a DataFrame for further analysis
df_bus_combined_geometry = df_bus_grouped_geometry.reset_index()
df_bus_combined_geometry.columns = ['ServiceNo', 'geometry']

# Step 4: Converting to a GPD
df_bus_combined_geometry = gpd.GeoDataFrame(
    df_bus_combined_geometry, geometry='geometry'
)

# Step 5: Setting the projection for the GPD
if df_bus_combined_geometry.crs is None:
    # Assuming the original CRS is EPSG:3414 (SVY21, common in Singapore)
    df_bus_combined_geometry = df_bus_combined_geometry.set_crs(epsg=4326) 

# Displaying the result
print(f"this is df_bus_combined_geometry:\n {df_bus_combined_geometry.head(5)}")

this is df_bus_combined_geometry:
   ServiceNo                                           geometry
0        10  LINESTRING (103.76868 1.29202, 103.76821 1.292...
1       100  LINESTRING (103.78715 1.31167, 103.78688 1.311...
2      100A  LINESTRING (103.87136 1.35037, 103.87167 1.348...
3       101  LINESTRING (103.87136 1.35037, 103.87163 1.348...
4       102  LINESTRING (103.86909 1.39288, 103.8691 1.3934...


# Processing LTA Datamall MRT Geospatial Data 
Goal: To have a geometry object representing every MRT Line in Singapore and concat to `df_bus_combined_geometry` (that contains the geometry object of each bus service number)
This is so that we are able to calculate the parallelism of bus routes with mrt lines 

Steps:
1) Setting the Coordinate Reference System for the LTA Datamall MRT Geospatial Data
2) Categorise MRT Stations into its respective lines and order them in right sequence (so that we can connect MRT Stations)
3) Every station in LTA Datamall's MRT Geospatial Data is represented by Polygon(s)
4) For each MRT_Line, connect the centroid of each station's Polygon(s) to form a "Line"
4) Within each MRT_Line, union the "Line" and all the Polygon(s) of all stations to form a singular geometry object representing the MRT_Line 

In [6]:
# Convert GeoDataFrame's geometry to latitude and longitude coordinates
if mrt_gdf.crs is None:
    # Assuming the original CRS is EPSG:3414 (SVY21, common in Singapore)
    mrt_gdf = mrt_gdf.set_crs(epsg=3414) 
mrt_gdf = mrt_gdf.set_geometry('geometry').to_crs(epsg=4326)
mrt_gdf['centroid'] = mrt_gdf['geometry'].centroid
mrt_gdf['lat'] = mrt_gdf['centroid'].y
mrt_gdf['lon'] = mrt_gdf['centroid'].x

C:\Users\brand\AppData\Local\Temp\ipykernel_12116\4108322666.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  mrt_gdf['centroid'] = mrt_gdf['geometry'].centroid


In [7]:
def get_ordered_mrt_lines_mapping(mrt_lines_mapping):
    # Drop redundant rows
    filtered_mrt_lines = mrt_lines_mapping[(mrt_lines_mapping['Station name_English • Malay'] != "north–south line (nsl)") & (mrt_lines_mapping['Station name_English • Malay'] != "downtown line (dtl)")].copy()
    # Group by each MRT Line, order them by the correct sequence
    filtered_mrt_lines['order'] = filtered_mrt_lines.groupby('MRT_Lines').cumcount() + 1
    # Remove duplicate MRT stations in each MRT Line
    filtered_mrt_lines = filtered_mrt_lines.sort_values('order').drop_duplicates(subset=['MRT_Lines', 'Station name_English • Malay'], keep='first')
    return filtered_mrt_lines

ordered_mrt_lines_mapping = get_ordered_mrt_lines_mapping(mrt_lines_mapping)
ordered_mrt_lines_mapping

,Station name_English • Malay,MRT_Lines,NS,EW,DT,CC,NE,TE,CG,CE,order
0,North–South Line (NSL),NS,1,0,0,0,0,0,0,0,1
28,Pasir Ris,EW,0,1,0,0,0,0,0,0,1
61,Expo,CG,0,0,1,0,0,0,1,0,1
63,HarbourFront,NE,0,0,0,1,1,0,0,0,1
108,Bayfront,CE,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
59,Tuas West Road,EW,0,1,0,0,0,0,0,0,32
60,Tuas Link,EW,0,1,0,0,0,0,0,0,33
142,Tampines East,DT,0,0,1,0,0,0,0,0,33
143,Upper Changi,DT,0,0,1,0,0,0,0,0,34


In [8]:
# Function to filter out "LRT", remove "MRT STATION", fuzzy match, and join MRT lines, then drop unwanted columns
def match_mrt_lines(ordered_mrt_lines_mapping, mrt_gdf, threshold=50):
    # Step 1: Filter out LRTs and 'DEPOT' stations
    filtered_mrt_gdf = mrt_gdf[(mrt_gdf['TYP_CD_DES'] == "MRT") & (~mrt_gdf['STN_NAM_DE'].str.contains('DEPOT', case=False))].copy()

    # Step 2: Remove "MRT STATION" from 'STN_NAM_DE' before matching and convert to lower case
    filtered_mrt_gdf['stn_name_cleaned'] = filtered_mrt_gdf['STN_NAM_DE'].str.replace('MRT STATION', '', case=False).str.strip()

    # Step 3: Perform fuzzy matching and create a new column for the best matched station name
    # Get the MRT station names in lowercase for case-insensitive matching
    mrt_station_names = ordered_mrt_lines_mapping['Station name_English • Malay'].str.lower().tolist()
    
    matched_station_names = []  # List to store matched station names

    for idx, row in filtered_mrt_gdf.iterrows():
        mrt_name = row['stn_name_cleaned'].lower()  # Lowercase the cleaned station name for case-insensitive matching
        
        # Perform fuzzy matching to find the closest MRT station name
        match_result = process.extractOne(mrt_name, mrt_station_names, scorer=fuzz.token_sort_ratio)
        
        if match_result:
            match, score, _ = match_result
            if score >= threshold:
                matched_station_names.append(match)
            else:
                matched_station_names.append(None)
        else:
            matched_station_names.append(None)
    
    # Add the matched station names to the GeoDataFrame as a new column
    filtered_mrt_gdf['stn_name_matched'] = matched_station_names
    
    # Step 4: Perform a left join to the MRT lines mapping DataFrame
    # Standardize the MRT station names to lowercase for the merge
    ordered_mrt_lines_mapping['Station name_English • Malay'] = ordered_mrt_lines_mapping['Station name_English • Malay'].str.lower()

    # Left join based on the matched station names
    merged_df = filtered_mrt_gdf.merge(ordered_mrt_lines_mapping, 
                                        left_on='stn_name_matched', 
                                        right_on='Station name_English • Malay', 
                                        how='left')

    # Step 5: Drop the unnecessary columns
    cols_to_drop = ['Matched_MRT_Line', 'stn_name_matched','stn_name_cleaned']
    merged_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

    # Step 6: Stations to drop as these are currently under construction or not accessible to public e.g. HALIFAX SUB STATION CABLE TROUGH, FOUNDERS' MEMORIAL MRT STATION, BUKIT BROWN MRT STATION, MOUNT PLEASANT MRT STATION
    # print([merged_df['MRT_Lines'].isna()]['STN_NAM_DE'])

    final_df = merged_df.dropna(subset=['MRT_Lines'])
    final_df = final_df[(final_df['STN_NAM_DE'] != "BUKIT BROWN MRT STATION") & (final_df['STN_NAM_DE'] != "MOUNT PLEASANT MRT STATION") & (final_df['STN_NAM_DE'] != "SUB STATION")]

    return final_df

mrt_gdf_processed = match_mrt_lines(ordered_mrt_lines_mapping, mrt_gdf)
mrt_gdf_processed

,ATTACHEMEN,STN_NAM,STN_NAM_DE,TYP_CD,TYP_CD_DES,geometry,centroid,lat,lon,Station name_English • Malay,MRT_Lines,NS,EW,DT,CC,NE,TE,CG,CE,order
0,None,None,HILLVIEW MRT STATION,0,MRT,"POLYGON ((103.76728 1.36249, 103.76732 1.36263...",POINT (103.76743 1.36234),1.362340,103.767429,hillview,DT,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0
1,None,None,BEAUTY WORLD MRT STATION,0,MRT,"POLYGON ((103.77576 1.34079, 103.77567 1.34077...",POINT (103.77582 1.3412),1.341204,103.775819,beauty world,DT,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0
3,None,None,BUKIT PANJANG MRT STATION,0,MRT,"POLYGON ((103.7614 1.37971, 103.76169 1.37917,...",POINT (103.76157 1.37916),1.379160,103.761573,bukit panjang,DT,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
4,None,None,CASHEW MRT STATION,0,MRT,"POLYGON ((103.76449 1.37021, 103.76462 1.37001...",POINT (103.7647 1.36936),1.369362,103.764697,cashew,DT,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
5,None,None,DHOBY GHAUT MRT STATION,0,MRT,"POLYGON ((103.84494 1.29945, 103.84554 1.29926...",POINT (103.84583 1.29904),1.299044,103.845833,dhoby ghaut,CC,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,None,None,TANJONG KATONG MRT STATION,0,MRT,"POLYGON ((103.89731 1.29902, 103.89733 1.29908...",POINT (103.89745 1.29936),1.299364,103.897451,tanjong katong,TE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,23.0
232,None,None,KATONG PARK MRT STATION,0,MRT,"POLYGON ((103.88506 1.29839, 103.8852 1.29838,...",POINT (103.88618 1.29782),1.297822,103.886182,katong park,TE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22.0
233,None,None,MARINE TERRACE MRT STATION,0,MRT,"POLYGON ((103.91629 1.30668, 103.91629 1.30669...",POINT (103.91532 1.30679),1.306786,103.915316,marine terrace,TE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,25.0
234,None,None,TANJONG RHU MRT STATION,0,MRT,"POLYGON ((103.87319 1.29647, 103.87317 1.29704...",POINT (103.87345 1.29721),1.297215,103.873452,tanjong rhu,TE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,21.0


In [38]:
def connect_mrt_stations(mrt_line, mrt_gdf_processed, buffer_distance):
     # Filter the DataFrame by the specified MRT line
    filtered_gdf = mrt_gdf_processed[mrt_gdf_processed['MRT_Lines'] == mrt_line].copy()

    # Group by 'STN_NAM_DE' and calculate the centroid of centroids if there are multiple rows of Polygons per station
    aggregated_df = (
        filtered_gdf.groupby('STN_NAM_DE', as_index=False)
        .agg({'centroid': lambda x: Point(sum([p.x for p in x]) / len(x), 
                                          sum([p.y for p in x]) / len(x)),
              'order': 'first'})  # Keep the first 'order' for each station
    )

    # Sort the aggregated DataFrame by the 'order' column
    sorted_df = aggregated_df.sort_values(by='order')

    # Extract the final centroids in the correct order
    final_centroids = sorted_df['centroid'].tolist()

    # Create a LineString from the final centroids
    line_geometry = LineString(final_centroids)

    if mrt_line == 'EW':
        # Filter to specifically get Tanah Merah mrt station
        tanah_merah_gdf = mrt_gdf_processed[(mrt_gdf_processed['STN_NAM_DE'] == 'TANAH MERAH MRT STATION') & (mrt_gdf_processed['MRT_Lines'] == 'EW')].copy()

        # Group by 'STN_NAM_DE' and calculate the centroid of centroids if there are multiple rows of Polygons for Tanah Merah station
        tanah_merah_aggregated_df = (
            tanah_merah_gdf.groupby('STN_NAM_DE', as_index=False)
            .agg({'centroid': lambda x: Point(sum([p.x for p in x]) / len(x), 
                                            sum([p.y for p in x]) / len(x)),
                'order': 'first'})  # Keep the first 'order' for each station
        )

        # Extract the centroid of Tanah Merah
        tanah_merah_final_centroids = tanah_merah_aggregated_df['centroid'].tolist()

        # Filter to get Expo and Changi Airport mrt station (part of the CG line)
        cg_gdf = mrt_gdf_processed[mrt_gdf_processed['MRT_Lines'] == 'CG'].copy()

        # Group by 'STN_NAM_DE' and calculate the centroid of centroids if there are multiple rows of Polygons for each stn in CG line
        cg_aggregated_df = (
            cg_gdf.groupby('STN_NAM_DE', as_index=False)
            .agg({'centroid': lambda x: Point(sum([p.x for p in x]) / len(x), 
                                            sum([p.y for p in x]) / len(x)),
                'order': 'first'})  # Keep the first 'order' for each station
        )

        # Sort the aggregated DataFrame by the 'order' column for CG line
        cg_sorted_df = cg_aggregated_df.sort_values(by='order')

        # Extract the final centroids in the correct order for CG line
        cg_centroids = cg_sorted_df['centroid'].tolist()

        # Add Tanah Merah centroid with CG line centroids (in this order)
        cg_final_centroids = tanah_merah_final_centroids + cg_centroids

        # Create a LineString from the final centroids
        cg_line_geometry = LineString(cg_final_centroids)

        # Final Line for EW line 
        line_geometry = unary_union([line_geometry, cg_line_geometry])
    
    elif mrt_line == 'CC':
        # Filter to specifically get Promenade mrt station
        promenade_gdf = mrt_gdf_processed[(mrt_gdf_processed['STN_NAM_DE'] == 'PROMENADE MRT STATION') & (mrt_gdf_processed['MRT_Lines'] == 'CC')].copy()

        # Group by 'STN_NAM_DE' and calculate the centroid of centroids if there are multiple rows of Polygons for Tanah Merah station
        promenade_aggregated_df = (
            promenade_gdf.groupby('STN_NAM_DE', as_index=False)
            .agg({'centroid': lambda x: Point(sum([p.x for p in x]) / len(x), 
                                            sum([p.y for p in x]) / len(x)),
                'order': 'first'})  # Keep the first 'order' for each station
        )

        # Extract the centroid of Promenade
        promenade_final_centroids = promenade_aggregated_df['centroid'].tolist()

        # Filter to get Bayfront and Marina Bay mrt station (part of the CE line)
        ce_gdf = mrt_gdf_processed[mrt_gdf_processed['MRT_Lines'] == 'CE'].copy()

        # Group by 'STN_NAM_DE' and calculate the centroid of centroids if there are multiple rows of Polygons for each stn in CE line
        ce_aggregated_df = (
            ce_gdf.groupby('STN_NAM_DE', as_index=False)
            .agg({'centroid': lambda x: Point(sum([p.x for p in x]) / len(x), 
                                            sum([p.y for p in x]) / len(x)),
                'order': 'first'})  # Keep the first 'order' for each station
        )

        # Sort the aggregated DataFrame by the 'order' column for CE line
        ce_sorted_df = ce_aggregated_df.sort_values(by='order')

        # Extract the final centroids in the correct order for CE line
        ce_centroids = ce_sorted_df['centroid'].tolist()

        # Add Promenade centroid with CG line centroids (in this order)
        ce_final_centroids = promenade_final_centroids + ce_centroids

        # Create a LineString from the final centroids
        ce_line_geometry = LineString(ce_final_centroids)

        # Final Line for CC line 
        line_geometry = unary_union([line_geometry, ce_line_geometry])
    
    # Create a transformer function to convert coordinates
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    def project(x, y):
        return transformer.transform(x, y)

    # Use shapely.ops.transform to apply the transformation to the LineString
    line_projected = transform(project, line_geometry)

    # Buffer the LineString (e.g., 200 meters)
    buffered_line = line_projected.buffer(buffer_distance)
    
    # Transform the buffered polygon back to EPSG:4326
    buffered_polygon_back_to_epsg4326 = Polygon(
        [transformer.transform(x, y, direction="INVERSE") for x, y in buffered_line.exterior.coords]
    )

    # Create a GeoDataFrame to store the result
    result_gdf = gpd.GeoDataFrame(
        {'MRT_Lines': [mrt_line], 'geometry': [buffered_polygon_back_to_epsg4326]},
        crs=mrt_gdf_processed.crs
    )

    return result_gdf


def combine_geom_by_mrt_line(mrt_line, mrt_geom_gdf, mrt_gdf_processed): 
    # Filter the DataFrame by the specified MRT line
    filtered_gdf = mrt_gdf_processed[mrt_gdf_processed['MRT_Lines'] == mrt_line].copy()

    # Get the geometry col of filtered_gdf
    combined_geo_series = filtered_gdf['geometry']

    # Extract the LineString geometry from the LineString GeoDataFrame
    line = mrt_geom_gdf['geometry'].iloc[0]  # There's one LineString per MRT line

    if mrt_line == "EW":
        # Get Polygons of the CG line stations (that are actually part of EW Line)
        cg_filtered_gdf = mrt_gdf_processed[mrt_gdf_processed['MRT_Lines'] == 'CG'].copy()

        # Subset only the geom col
        cg_filtered_gdf_geom = cg_filtered_gdf['geometry']

        # Combine existing EW Line polygons with CG Line polygins into a single Geo Series
        combined_geo_series = pd.concat([combined_geo_series, cg_filtered_gdf_geom], ignore_index=False)
    
    elif mrt_line == "CC":
        # Get Polygons of the CE line stations (that are actually part of CC Line)
        ce_filtered_gdf = mrt_gdf_processed[mrt_gdf_processed['MRT_Lines'] == 'CE'].copy()

        # Subset only the geom col
        ce_filtered_gdf_geom = ce_filtered_gdf['geometry']

        # Combine existing EW Line polygons with CG Line polygins into a single Geo Series
        combined_geo_series = pd.concat([combined_geo_series, ce_filtered_gdf_geom], ignore_index=False)

    # Combine all Polygon geometries into a single unified MultiPolygon/Polygon
    unified_polygon = combined_geo_series.unary_union  # Merge all polygons

    # Perform the union of the unified polygon and the LineString
    combined_geometry = unified_polygon.union(line)

    # Create a GeoDataFrame to store the result
    result_gdf = gpd.GeoDataFrame(
        {'MRT_Lines': [mrt_line], 'geometry': [combined_geometry]},
        crs=mrt_gdf_processed.crs
    )
    
    return result_gdf 



# Connecting all the MRT Station for each MRT_Line to form a LineString  
ns_mrt_geom_gdf = connect_mrt_stations('NS', mrt_gdf_processed, 400)
ew_mrt_geom_gdf = connect_mrt_stations('EW', mrt_gdf_processed, 400)
dt_mrt_geom_gdf = connect_mrt_stations('DT', mrt_gdf_processed, 400)
cc_mrt_geom_gdf = connect_mrt_stations('CC', mrt_gdf_processed, 400)
ne_mrt_geom_gdf = connect_mrt_stations('NE', mrt_gdf_processed, 400)
te_mrt_geom_gdf = connect_mrt_stations('TE', mrt_gdf_processed, 400)

# Combining the LineString for each MRT_Line with the Polygons of each MRT Station
final_ns_mrt_geom_gdf = combine_geom_by_mrt_line('NS', ns_mrt_geom_gdf, mrt_gdf_processed)
final_ew_mrt_geom_gdf = combine_geom_by_mrt_line('EW', ew_mrt_geom_gdf, mrt_gdf_processed)
final_dt_mrt_geom_gdf = combine_geom_by_mrt_line('DT', dt_mrt_geom_gdf, mrt_gdf_processed)
final_cc_mrt_geom_gdf = combine_geom_by_mrt_line('CC', cc_mrt_geom_gdf, mrt_gdf_processed)
final_ne_mrt_geom_gdf = combine_geom_by_mrt_line('NE', ne_mrt_geom_gdf, mrt_gdf_processed)
final_te_mrt_geom_gdf = combine_geom_by_mrt_line('TE', te_mrt_geom_gdf, mrt_gdf_processed)

print(ew_mrt_geom_gdf)
print(final_ew_mrt_geom_gdf)

  MRT_Lines                                           geometry
0        EW  POLYGON ((103.93087 1.32049, 103.93078 1.32047...
  MRT_Lines                                           geometry
0        EW  POLYGON ((103.93078 1.32047, 103.91356 1.3175,...


C:\Users\brand\AppData\Local\Temp\ipykernel_12116\2430551732.py:160: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  unified_polygon = combined_geo_series.unary_union  # Merge all polygons
C:\Users\brand\AppData\Local\Temp\ipykernel_12116\2430551732.py:160: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  unified_polygon = combined_geo_series.unary_union  # Merge all polygons
C:\Users\brand\AppData\Local\Temp\ipykernel_12116\2430551732.py:160: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  unified_polygon = combined_geo_series.unary_union  # Merge all polygons
C:\Users\brand\AppData\Local\Temp\ipykernel_12116\2430551732.py:160: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  unified_polygon = combined_geo_series.unary_union  # Merge all polygons
C:\Users\brand\AppData\L

## Concat each mrt_line geom obj to the `df_bus_combined_geometry` (that contains the geometry object of each bus service number)

In [39]:
# Adding each mrt_line geom obj to the bus_service gdf
bus_mrt_combined_gdf = df_bus_combined_geometry.copy()
bus_mrt_combined_gdf['NS_MRT_geom'] = final_ns_mrt_geom_gdf.iloc[0]['geometry']
bus_mrt_combined_gdf['EW_MRT_geom'] = final_ew_mrt_geom_gdf.iloc[0]['geometry']
bus_mrt_combined_gdf['DT_MRT_geom'] = final_dt_mrt_geom_gdf.iloc[0]['geometry']
bus_mrt_combined_gdf['CC_MRT_geom'] = final_cc_mrt_geom_gdf.iloc[0]['geometry']
bus_mrt_combined_gdf['NE_MRT_geom'] = final_ne_mrt_geom_gdf.iloc[0]['geometry']
bus_mrt_combined_gdf['TE_MRT_geom'] = final_te_mrt_geom_gdf.iloc[0]['geometry']

bus_mrt_combined_gdf

,ServiceNo,geometry,NS_MRT_geom,EW_MRT_geom,DT_MRT_geom,CC_MRT_geom,NE_MRT_geom,TE_MRT_geom
0,10,"LINESTRING (103.76868 1.29202, 103.76821 1.292...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...
1,100,"LINESTRING (103.78715 1.31167, 103.78688 1.311...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...
2,100A,"LINESTRING (103.87136 1.35037, 103.87167 1.348...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...
3,101,"LINESTRING (103.87136 1.35037, 103.87163 1.348...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...
4,102,"LINESTRING (103.86909 1.39288, 103.8691 1.3934...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...
...,...,...,...,...,...,...,...,...
551,991C,"LINESTRING (103.7363 1.37665, 103.73601 1.3764...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...
552,992,"LINESTRING (103.72925 1.3575, 103.73042 1.3562...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...
553,993,"LINESTRING (103.73549 1.34197, 103.73577 1.342...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...
554,9A,"LINESTRING (103.92899 1.32472, 103.92868 1.324...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...


In [40]:
# Exporting for Front End and future modelling to use
bus_mrt_combined_gdf.to_pickle("data/bus_mrt_combined_gdf.pkl")

In [41]:
def buffer_bus_route(bus_mrt_combined_gdf, buffer_distance, new_col="buffered_bus_route_geom"):
    """
    Buffers the 'geometry' column (corresponds to geom for bus routes) of the bus_mrt_combined GeoDataFrame by the specified buffer distance (in meters).
    Args:
        bus_mrt_combined_gdf (GeoDataFrame): The input GeoDataFrame with a 'geometry' column.
        buffer_distance (float): The distance (in meters) to buffer the geometries.

    Returns:
        GeoDataFrame: A new GeoDataFrame with an additional 'buffered_geometry' column.
    """
    # Step 1: Reproject to EPSG:3857 (Web Mercator) to apply buffering in meters
    bus_mrt_combined_gdf_projected = bus_mrt_combined_gdf.to_crs(epsg=3857)

    # Step 2:  Create a new column and Apply the buffer in meters
    bus_mrt_combined_gdf_projected[new_col] = bus_mrt_combined_gdf_projected['geometry'].buffer(buffer_distance)

    # Step 3: Reproject back to EPSG: 4326 (Original)
    bus_mrt_combined_gdf_buffered = bus_mrt_combined_gdf_projected.to_crs(epsg=4326)
    bus_mrt_combined_gdf_buffered = bus_mrt_combined_gdf_buffered.rename(columns={'geometry': 'original_bus_route_geom'})
    bus_mrt_combined_gdf_buffered = bus_mrt_combined_gdf_buffered.set_geometry(new_col)
    bus_mrt_combined_gdf_buffered = bus_mrt_combined_gdf_buffered.to_crs(epsg=4326)
    
    return bus_mrt_combined_gdf_buffered

buffered_bus_mrt_combined_gdf = buffer_bus_route(bus_mrt_combined_gdf, 400)
buffered_bus_mrt_combined_gdf

,ServiceNo,original_bus_route_geom,NS_MRT_geom,EW_MRT_geom,DT_MRT_geom,CC_MRT_geom,NE_MRT_geom,TE_MRT_geom,buffered_bus_route_geom
0,10,"LINESTRING (103.76868 1.29202, 103.76821 1.292...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.76596 1.29579, 103.76598 1.29629..."
1,100,"LINESTRING (103.78715 1.31167, 103.78688 1.311...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.78204 1.31309, 103.78204 1.31311..."
2,100A,"LINESTRING (103.87136 1.35037, 103.87167 1.348...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.86784 1.34646, 103.86805 1.34699..."
3,101,"LINESTRING (103.87136 1.35037, 103.87163 1.348...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.87278 1.35408, 103.87483 1.35604..."
4,102,"LINESTRING (103.86909 1.39288, 103.8691 1.3934...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.86533 1.40187, 103.86501 1.40198..."
...,...,...,...,...,...,...,...,...,...
551,991C,"LINESTRING (103.7363 1.37665, 103.73601 1.3764...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.73984 1.37725, 103.74025 1.37699..."
552,992,"LINESTRING (103.72925 1.3575, 103.73042 1.3562...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.73244 1.35314, 103.73242 1.35313..."
553,993,"LINESTRING (103.73549 1.34197, 103.73577 1.342...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.73249 1.34396, 103.7325 1.34397,..."
554,9A,"LINESTRING (103.92899 1.32472, 103.92868 1.324...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.93077 1.32784, 103.93257 1.32796..."


In [42]:
# Exporting for Front End and future modelling to use
buffered_bus_mrt_combined_gdf.to_pickle("data/buffered_bus_mrt_combined_gdf.pkl")

### Function to visualise a particular bus service together with all the MRT Lines (unbuffered bus route)

In [14]:
def plot_mrt_combined_geometry(mrt_geom_gdf):
    # Initialize a Folium map centered at the approximate center of the geometry
    centroid = mrt_geom_gdf['geometry'].iloc[0].centroid
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=13)

    # Add the geometry to the map
    folium.GeoJson(
        mapping(mrt_geom_gdf['geometry'].iloc[0]),  # Convert geometry to GeoJSON-like mapping
        name="Combined Geometry"
    ).add_to(m)

    # Add a layer control to toggle layers
    folium.LayerControl().add_to(m)

    return m

plot_mrt_combined_geometry(final_cc_mrt_geom_gdf)

In [15]:
def plot_bus_service_and_mrt_routes(service_no, gdf):
    """
    Plots the 'geometry' route and all MRT geometries for a given ServiceNo using Folium.
    
    Parameters:
    - service_no: str, the service number to search for
    - gdf: GeoDataFrame, contains the bus and MRT routes data
    
    Returns:
    - A Folium map centered on the bus route or MRT geometry.
    """
    # Filter the row with the given service number
    row = gdf[gdf['ServiceNo'] == service_no]
    
    if row.empty:
        print(f"No data found for ServiceNo: {service_no}")
        return None

    # Extract the geometries (assume each is a GeoSeries)
    bus_route = row.iloc[0]['geometry']
    mrt_geoms = [
        row.iloc[0]['NS_MRT_geom'],
        row.iloc[0]['EW_MRT_geom'],
        row.iloc[0]['DT_MRT_geom'],
        row.iloc[0]['CC_MRT_geom'],
        row.iloc[0]['NE_MRT_geom'],
        row.iloc[0]['TE_MRT_geom']
    ]

    # Corresponding MRT line colors
    colors = [
        'red',         # NS Line
        'green',       # EW Line
        'darkblue',    # DT Line
        'yellow',      # CC Line
        'purple',      # NE Line
        'brown'        # TE Line
    ]

    # Center the map on the first point of the bus route or MRT geometry
    # start_coords = [bus_route.coords[0][1], bus_route.coords[0][0]]  # lat, lon
    start_coords = [1.3521, 103.8198]

    # Create a Folium map centered on the first point of the bus route
    m = folium.Map(location=start_coords, zoom_start=13)

    # Plot the bus route in black
    folium.GeoJson(
        bus_route,
        name='Bus Route',
        style_function=lambda x: {'color': 'black', 'weight': 3}
    ).add_to(m)

    # Plot each MRT line geometry with the corresponding color
    for mrt_geom, color, name in zip(mrt_geoms, colors, [
        'NS Line', 'EW Line', 'DT Line', 'CC Line', 'NE Line', 'TE Line'
    ]):
        folium.GeoJson(
            mrt_geom,
            name=name,
            style_function=lambda x, color=color: {'color': color, 'weight': 2}
        ).add_to(m)

    # Add a layer control to switch between routes
    folium.LayerControl().add_to(m)

    # Display the map
    return m
plot_bus_service_and_mrt_routes('35', bus_mrt_combined_gdf)

In [43]:
def modified_plot_bus_service_and_mrt_routes(service_no, gdf):
    """
    Plots the 'geometry' route and all MRT geometries for a given ServiceNo using Folium.
    
    Parameters:
    - service_no: str, the service number to search for
    - gdf: GeoDataFrame, contains the bus and MRT routes data
    
    Returns:
    - A Folium map centered on the bus route or MRT geometry.
    """
    # Filter the row with the given service number
    row = gdf[gdf['ServiceNo'] == service_no]

    if row.empty:
        print(f"No data found for ServiceNo: {service_no}")
        return None

    # Extract the buffered bus route geometry
    bus_route = row.iloc[0]['buffered_bus_route_geom']

    # Extract the MRT line geometries
    mrt_geoms = [
        row.iloc[0]['NS_MRT_geom'],
        row.iloc[0]['EW_MRT_geom'],
        row.iloc[0]['DT_MRT_geom'],
        row.iloc[0]['CC_MRT_geom'],
        row.iloc[0]['NE_MRT_geom'],
        row.iloc[0]['TE_MRT_geom']
    ]

    # Corresponding MRT line colors
    colors = [
        'red',         # NS Line
        'green',       # EW Line
        'darkblue',    # DT Line
        'yellow',      # CC Line
        'purple',      # NE Line
        'brown'        # TE Line
    ]

    # Center the map on the first point of the buffered bus route or MRT geometry
    # start_coords = [bus_route.centroid.y, bus_route.centroid.x]  # lat, lon
    start_coords = [1.3521, 103.8198]

    # Create a Folium map centered on the bus route
    m = folium.Map(location=start_coords, zoom_start=13)

    # Plot the buffered bus route in black
    folium.GeoJson(
        bus_route,
        name='Buffered Bus Route',
        style_function=lambda x: {'color': 'black', 'weight': 2, 'fillOpacity': 0.3}
    ).add_to(m)

    # Plot each MRT line geometry with the corresponding color
    for mrt_geom, color, name in zip(mrt_geoms, colors, [
        'NS Line', 'EW Line', 'DT Line', 'CC Line', 'NE Line', 'TE Line'
    ]):
        folium.GeoJson(
            mrt_geom,
            name=name,
            style_function=lambda x, color=color: {'color': color, 'weight': 2}
        ).add_to(m)

    # Add a layer control to switch between routes
    folium.LayerControl().add_to(m)

    # Return the map object
    return m
modified_plot_bus_service_and_mrt_routes('883', buffered_bus_mrt_combined_gdf)

# Feature 1: Overlap percentage between bus service and mrt lines
## Measure of Bus Service Parallelism with MRT Lines

In [17]:
def calculate_overlap_percentage(row, mrt_columns):
    # Extract the Bus ServiceNo.
    bus_service_no = row['ServiceNo']
    bus_geom = row['buffered_bus_route_geom']
    bus_area = bus_geom.area  # Total area of the buffered bus route

    overlap_percentages = {}
    overlap_percentages = {'ServiceNo': bus_service_no}

    for mrt_line in mrt_columns:
        mrt_geom = row[mrt_line]

        # compute area of intersection
        intersection = bus_geom.intersection(mrt_geom) 
        intersection_area = intersection.area if not intersection.is_empty else 0

        # calculate overlap percentage
        overlap_percentage = (intersection_area / bus_area) * 100 if bus_area > 0 else 0
        overlap_percentages[mrt_line] = overlap_percentage

    return overlap_percentages

def apply_overlap_calculations(gdf, mrt_columns):

    # Apply the calculation row by row
    overlap_results = gdf.apply(lambda row: calculate_overlap_percentage(row, mrt_columns), axis=1)

    # convert the list of dictionaries into a DataFrame
    overlap_df = overlap_results.apply(pd.Series)

    # Concatenate with the original GeoDataFrame 
    return pd.concat([gdf, overlap_df], axis=1)

mrt_columns = [
    'NS_MRT_geom', 'EW_MRT_geom', 'DT_MRT_geom', 'CC_MRT_geom', 'NE_MRT_geom', 'TE_MRT_geom'
]

bus_mrt_combined_area_gdf = apply_overlap_calculations(buffered_bus_mrt_combined_gdf, mrt_columns)

bus_mrt_combined_area_gdf

,ServiceNo,original_bus_route_geom,NS_MRT_geom,EW_MRT_geom,DT_MRT_geom,CC_MRT_geom,NE_MRT_geom,TE_MRT_geom,buffered_bus_route_geom,ServiceNo,NS_MRT_geom,EW_MRT_geom,DT_MRT_geom,CC_MRT_geom,NE_MRT_geom,TE_MRT_geom
0,10,"LINESTRING (103.76868 1.29202, 103.76821 1.292...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.76596 1.29579, 103.76598 1.29629...",10,5.227974,20.959093,13.498189,30.487208,5.731992,13.614954
1,100,"LINESTRING (103.78715 1.31167, 103.78688 1.311...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.78204 1.31309, 103.78204 1.31311...",100,6.848529,34.687877,17.601992,28.508462,15.214120,3.682970
2,100A,"LINESTRING (103.87136 1.35037, 103.87167 1.348...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.86784 1.34646, 103.86805 1.34699...",100A,0.000000,13.287873,15.697995,18.887485,36.216515,0.000000
3,101,"LINESTRING (103.87136 1.35037, 103.87163 1.348...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.87278 1.35408, 103.87483 1.35604...",101,0.000000,0.000000,0.000000,13.940347,55.021526,0.000000
4,102,"LINESTRING (103.86909 1.39288, 103.8691 1.3934...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.86533 1.40187, 103.86501 1.40198...",102,0.000000,0.000000,0.000000,0.000000,24.791770,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,991C,"LINESTRING (103.7363 1.37665, 103.73601 1.3764...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.73984 1.37725, 103.74025 1.37699...",991C,54.928368,0.000000,0.000000,0.000000,0.000000,0.000000
552,992,"LINESTRING (103.72925 1.3575, 103.73042 1.3562...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.73244 1.35314, 103.73242 1.35313...",992,16.369616,0.000000,0.000000,0.000000,0.000000,0.000000
553,993,"LINESTRING (103.73549 1.34197, 103.73577 1.342...",POLYGON ((103.74819386280069 1.358603078202061...,POLYGON ((103.93078388188964 1.320471391426507...,POLYGON ((103.77077365658603 1.363653731665758...,POLYGON ((103.85248916520659 1.299916164460304...,POLYGON ((103.83708534828865 1.283642218478883...,POLYGON ((103.7967313259274 1.4293047118045055...,"POLYGON ((103.73249 1.34396, 103.7325 1.34397,...",993,13.6

In [ ]:
bus_mrt_combined_area_gdf_subset = bus_mrt_combined_area_gdf.iloc[:, -7:].copy()
bus_mrt_combined_area_gdf_subset = bus_mrt_combined_area_gdf_subset.rename(columns={
    "NS_MRT_geom" : "NS_MRT_Overlap_Pct",
    "EW_MRT_geom": "EW_MRT_Overlap_Pct", 
    "DT_MRT_geom": "DT_MRT_Overlap_Pct", 
    "CC_MRT_geom": "CC_MRT_Overlap_Pct", 
    "NE_MRT_geom": "NE_MRT_Overlap_Pct", 
    "TE_MRT_geom": "TE_MRT_Overlap_Pct"
})
bus_mrt_combined_area_gdf_subset

,ServiceNo,NS_MRT_Overlap_Pct,EW_MRT_Overlap_Pct,DT_MRT_Overlap_Pct,CC_MRT_Overlap_Pct,NE_MRT_Overlap_Pct,TE_MRT_Overlap_Pct
0,10,5.227974,20.959093,13.498189,30.487208,5.731992,13.614954
1,100,6.848529,34.687877,17.601992,28.508462,15.214120,3.682970
2,100A,0.000000,13.287873,15.697995,18.887485,36.216515,0.000000
3,101,0.000000,0.000000,0.000000,13.940347,55.021526,0.000000
4,102,0.000000,0.000000,0.000000,0.000000,24.791770,0.000000
...,...,...,...,...,...,...,...
551,991C,54.928368,0.000000,0.000000,0.000000,0.000000,0.000000
552,992,16.369616,0.000000,0.000000,0.000000,0.000000,0.000000
553,993,13.656012,33.195281,0.000000,0.000000,0.000000,0.000000
554,9A,0.000000,57.458946,20.847763,0.000000,0.000000,0.000000


In [36]:
bus_mrt_combined_area_gdf_subset.to_csv("data/bus_mrt_overlap_pct.csv", index=False)

# Feature 2: Passenger Volume Origin-Destination Bus Stops
## Measure of demand for bus services

In [18]:
def aggregate_od_volume_df(od_volume_df):
    aggregated_df = (
        od_volume_df
        .groupby(['DAY_TYPE', 'ORIGIN_PT_CODE', 'DESTINATION_PT_CODE'], as_index=False)
        .agg(AGGREGATED_TOTAL_TRIPS=('TOTAL_TRIPS', 'sum'))
    )
    
    return aggregated_df


def calculate_total_trips(bus_routes_df: pd.DataFrame, aggregated_od_volume_df: pd.DataFrame) -> pd.DataFrame:
    # Group by ServiceNo and Direction, sort by StopSequence
    grouped_df = (
        bus_routes_df
        .sort_values(by=['ServiceNo', 'Direction', 'StopSequence'])
        .groupby(['ServiceNo', 'Direction'])
    )
    
    # Prepare lists to store the result
    service_no_list = []
    direction_list = []
    origin_bus_stops_lst = []
    destination_bus_stops_lst = []
    weekday_trips_list = []
    weekend_trips_list = []
    
    # Iterate through each group (ServiceNo, Direction)
    for (service_no, direction), group in grouped_df:
        # Get the stop codes in order of StopSequence
        stop_codes = group['BusStopCode'].values
        
        # Iterate over pairs of consecutive stop codes
        for i in range(len(stop_codes) - 1):
            origin_pt_code = stop_codes[i]
            destination_pt_code = stop_codes[i + 1]
            
            # Filter for WEEKDAY trips
            weekday_trips = aggregated_od_volume_df[
                (aggregated_od_volume_df['DAY_TYPE'] == 'WEEKDAY') &
                (aggregated_od_volume_df['ORIGIN_PT_CODE'] == origin_pt_code) &
                (aggregated_od_volume_df['DESTINATION_PT_CODE'] == destination_pt_code)
            ]['AGGREGATED_TOTAL_TRIPS'].sum()
            
            # Filter for WEEKENDS/HOLIDAY trips
            weekend_trips = aggregated_od_volume_df[
                (aggregated_od_volume_df['DAY_TYPE'] == 'WEEKENDS/HOLIDAY') &
                (aggregated_od_volume_df['ORIGIN_PT_CODE'] == origin_pt_code) &
                (aggregated_od_volume_df['DESTINATION_PT_CODE'] == destination_pt_code)
            ]['AGGREGATED_TOTAL_TRIPS'].sum()
            
            # Store the results
            service_no_list.append(service_no)
            direction_list.append(direction)
            origin_bus_stops_lst.append(origin_pt_code)
            destination_bus_stops_lst.append(destination_pt_code)
            weekday_trips_list.append(weekday_trips)
            weekend_trips_list.append(weekend_trips)
    
    # Create the result DataFrame
    result_df = pd.DataFrame({
        'ServiceNo': service_no_list,
        'Direction': direction_list,
        'ORIGIN_PT_CODE': origin_bus_stops_lst,
        'DESTINATION_PT_CODE': destination_bus_stops_lst,
        'WEEKDAY_TOTAL_TRIPS': weekday_trips_list,
        'WEEKEND_TOTAL_TRIPS': weekend_trips_list
    })
    
    return result_df

def group_and_aggregate_trips(df: pd.DataFrame) -> pd.DataFrame:
    # Group by 'ServiceNo' and 'Direction', and aggregate with sum
    result = df.groupby(['ServiceNo'], as_index=False).agg(
        WEEKDAY_TOTAL_TRIPS=('WEEKDAY_TOTAL_TRIPS', 'sum'),
        WEEKEND_TOTAL_TRIPS=('WEEKEND_TOTAL_TRIPS', 'sum')
    )
    
    return result

In [ ]:
# OD Passenger Volume Data for July 2024
jul24_aggregated_od_volume_df = aggregate_od_volume_df(jul24_od_volume_df)
jul24_aggregated_od_volume_df

jul24_bus_journey_od_passenger_volume_df = calculate_total_trips(bus_routes_df, jul24_aggregated_od_volume_df)
jul24_bus_journey_od_passenger_volume_df

jul24_bus_service_od_passenger_volume_df = group_and_aggregate_trips(jul24_bus_journey_od_passenger_volume_df)
jul24_bus_service_od_passenger_volume_df

# OD Passenger Volume Data for August 2024
aug24_aggregated_od_volume_df = aggregate_od_volume_df(aug24_od_volume_df)
aug24_aggregated_od_volume_df

aug24_bus_journey_od_passenger_volume_df = calculate_total_trips(bus_routes_df, aug24_aggregated_od_volume_df)
aug24_bus_journey_od_passenger_volume_df

aug24_bus_service_od_passenger_volume_df = group_and_aggregate_trips(aug24_bus_journey_od_passenger_volume_df)
aug24_bus_service_od_passenger_volume_df

# OD Passenger Volume Data for September 2024
sep24_aggregated_od_volume_df = aggregate_od_volume_df(sep24_od_volume_df)
sep24_aggregated_od_volume_df

sep24_bus_journey_od_passenger_volume_df = calculate_total_trips(bus_routes_df, sep24_aggregated_od_volume_df)
sep24_bus_journey_od_passenger_volume_df

sep24_bus_service_od_passenger_volume_df = group_and_aggregate_trips(sep24_bus_journey_od_passenger_volume_df)
sep24_bus_service_od_passenger_volume_df

In [ ]:
jul24_bus_service_od_passenger_volume_df.rename(columns={'WEEKDAY_TOTAL_TRIPS': 'Jul24_WEEKDAY_TOTAL_TRIPS', 'WEEKEND_TOTAL_TRIPS': 'Jul24_WEEKEND_PH_TOTAL_TRIPS'}, inplace=True)
aug24_bus_service_od_passenger_volume_df.rename(columns={'WEEKDAY_TOTAL_TRIPS': 'Aug24_WEEKDAY_TOTAL_TRIPS', 'WEEKEND_TOTAL_TRIPS': 'Aug24_WEEKEND_PH_TOTAL_TRIPS'}, inplace=True)
sep24_bus_service_od_passenger_volume_df.rename(columns={'WEEKDAY_TOTAL_TRIPS': 'Sep24_WEEKDAY_TOTAL_TRIPS', 'WEEKEND_TOTAL_TRIPS': 'Sep24_WEEKEND_PH_TOTAL_TRIPS'}, inplace=True)

In [ ]:
jul24_bus_service_od_passenger_volume_df.to_csv("data/jul24_bus_service_od_passenger_volume_df.csv", index=False)
aug24_bus_service_od_passenger_volume_df.to_csv("data/aug24_bus_service_od_passenger_volume_df.csv", index=False)
sep24_bus_service_od_passenger_volume_df.to_csv("data/sep24_bus_service_od_passenger_volume_df.csv", index=False)

# Feature 3: Bus Routes Distance data
## To be used for clustering to identify residential trunks

In [20]:
def get_bus_routes_total_distance(bus_routes_df):
    # Group by 'ServiceNo' and 'Direction', and get the max 'Distance'
    total_distance_per_dir_df = (
        bus_routes_df
        .groupby(['ServiceNo', 'Direction'], as_index=False)
        .agg({'Distance': 'max'})
    )

    total_distance_df = (
        total_distance_per_dir_df
        .groupby(['ServiceNo'], as_index=False)
        .agg({'Distance':'sum'})
        .rename(columns={'Distance': 'Total_Distance'})
    )


    return total_distance_df

bus_routes_distance_df = get_bus_routes_total_distance(bus_routes_df)
bus_routes_distance_df

,ServiceNo,Total_Distance
0,10,63.5
1,100,48.0
2,100A,4.8
3,101,15.7
4,102,26.9
...,...,...
552,991C,3.0
553,992,11.9
554,993,9.2
555,9A,8.4


In [21]:
def get_bus_routes_average_distance(bus_routes_df):
    # Group by 'ServiceNo' and 'Direction', and get the max 'Distance'
    total_distance_per_dir_df = (
        bus_routes_df
        .groupby(['ServiceNo', 'Direction'], as_index=False)
        .agg({'Distance': 'max'})
    )

    average_distance_df = (
        total_distance_per_dir_df
        .groupby(['ServiceNo'], as_index=False)
        .agg({'Distance':'mean'})
        .rename(columns={'Distance': 'Average_Distance'})
    )


    return average_distance_df

bus_routes_avg_distance_df = get_bus_routes_average_distance(bus_routes_df)
bus_routes_avg_distance_df

,ServiceNo,Average_Distance
0,10,31.75
1,100,24.00
2,100A,4.80
3,101,15.70
4,102,26.90
...,...,...
552,991C,3.00
553,992,5.95
554,993,9.20
555,9A,8.40


In [22]:
bus_routes_avg_distance_df.to_csv("data/bus_routes_avg_distance_df.csv", index=False)

# Feature 4: Population Served
## Adding a demographic layer to our analysis as a measure of demand for each bus service

In [23]:
# Select the second column and the last 8 columns using indexing
columns_to_select = [1] + list(range(-8, 0))

# Apply the selection
bus_mrt_combined_gdf_overlap = bus_mrt_combined_area_gdf.iloc[:, columns_to_select]

# Move 'ServiceNo' to the first position by reordering the columns
columns = ['ServiceNo'] + [col for col in bus_mrt_combined_gdf_overlap.columns if col != 'ServiceNo']
bus_mrt_combined_gdf_overlap = bus_mrt_combined_gdf_overlap[columns]
bus_mrt_combined_gdf_overlap

,ServiceNo,original_bus_route_geom,buffered_bus_route_geom,NS_MRT_geom,EW_MRT_geom,DT_MRT_geom,CC_MRT_geom,NE_MRT_geom,TE_MRT_geom
0,10,"LINESTRING (103.76868 1.29202, 103.76821 1.292...","POLYGON ((103.76596 1.29579, 103.76598 1.29629...",5.227974,20.959093,13.498189,30.487208,5.731992,13.614954
1,100,"LINESTRING (103.78715 1.31167, 103.78688 1.311...","POLYGON ((103.78204 1.31309, 103.78204 1.31311...",6.848529,34.687877,17.601992,28.508462,15.214120,3.682970
2,100A,"LINESTRING (103.87136 1.35037, 103.87167 1.348...","POLYGON ((103.86784 1.34646, 103.86805 1.34699...",0.000000,13.287873,15.697995,18.887485,36.216515,0.000000
3,101,"LINESTRING (103.87136 1.35037, 103.87163 1.348...","POLYGON ((103.87278 1.35408, 103.87483 1.35604...",0.000000,0.000000,0.000000,13.940347,55.021526,0.000000
4,102,"LINESTRING (103.86909 1.39288, 103.8691 1.3934...","POLYGON ((103.86533 1.40187, 103.86501 1.40198...",0.000000,0.000000,0.000000,0.000000,24.791770,0.000000
...,...,...,...,...,...,...,...,...,...
551,991C,"LINESTRING (103.7363 1.37665, 103.73601 1.3764...","POLYGON ((103.73984 1.37725, 103.74025 1.37699...",54.928368,0.000000,0.000000,0.000000,0.000000,0.000000
552,992,"LINESTRING (103.72925 1.3575, 103.73042 1.3562...","POLYGON ((103.73244 1.35314, 103.73242 1.35313...",16.369616,0.000000,0.000000,0.000000,0.000000,0.000000
553,993,"LINESTRING (103.73549 1.34197, 103.73577 1.342...","POLYGON ((103.73249 1.34396, 103.7325 1.34397,...",13.656012,33.195281,0.000000,0.000000,0.000000,0.000000
554,9A,"LINESTRING (103.92899 1.32472, 103.92868 1.324...","POLYGON ((103.93077 1.32784, 103.93257 1.32796...",0.000000,57.458946,20.847763,0.000000,0.000000,0.000000


In [24]:
def categorise_bus_routes(bus_mrt_combined_gdf):
    # Extract unique bus services with their categories
    bus_services_info_subset_df = bus_services_info_df[['ServiceNo', 'Category']].drop_duplicates(subset='ServiceNo', keep='first')
    
    # Merge the bus MRT data with the bus service categories
    categorised_bus_mrt_combined_gdf = pd.merge(bus_mrt_combined_gdf, bus_services_info_subset_df, on='ServiceNo', how='left')
    
    return categorised_bus_mrt_combined_gdf 

## applying function to dataframe with bus_mrt_combined_overlap 
categorised_bus_mrt_combined_gdf_overlap = categorise_bus_routes(bus_mrt_combined_gdf_overlap)

categorised_bus_mrt_combined_gdf_overlap

,ServiceNo,original_bus_route_geom,buffered_bus_route_geom,NS_MRT_geom,EW_MRT_geom,DT_MRT_geom,CC_MRT_geom,NE_MRT_geom,TE_MRT_geom,Category
0,10,"LINESTRING (103.76868 1.29202, 103.76821 1.292...","POLYGON ((103.76596 1.29579, 103.76598 1.29629...",5.227974,20.959093,13.498189,30.487208,5.731992,13.614954,TRUNK
1,100,"LINESTRING (103.78715 1.31167, 103.78688 1.311...","POLYGON ((103.78204 1.31309, 103.78204 1.31311...",6.848529,34.687877,17.601992,28.508462,15.214120,3.682970,TRUNK
2,100A,"LINESTRING (103.87136 1.35037, 103.87167 1.348...","POLYGON ((103.86784 1.34646, 103.86805 1.34699...",0.000000,13.287873,15.697995,18.887485,36.216515,0.000000,TRUNK
3,101,"LINESTRING (103.87136 1.35037, 103.87163 1.348...","POLYGON ((103.87278 1.35408, 103.87483 1.35604...",0.000000,0.000000,0.000000,13.940347,55.021526,0.000000,TRUNK
4,102,"LINESTRING (103.86909 1.39288, 103.8691 1.3934...","POLYGON ((103.86533 1.40187, 103.86501 1.40198...",0.000000,0.000000,0.000000,0.000000,24.791770,0.000000,TRUNK
...,...,...,...,...,...,...,...,...,...,...
551,991C,"LINESTRING (103.7363 1.37665, 103.73601 1.3764...","POLYGON ((103.73984 1.37725, 103.74025 1.37699...",54.928368,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
552,992,"LINESTRING (103.72925 1.3575, 103.73042 1.3562...","POLYGON ((103.73244 1.35314, 103.73242 1.35313...",16.369616,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
553,993,"LINESTRING (103.73549 1.34197, 103.73577 1.342...","POLYGON ((103.73249 1.34396, 103.7325 1.34397,...",13.656012,33.195281,0.000000,0.000000,0.000000,0.000000,TRUNK
554,9A,"LINESTRING (103.92899 1.32472, 103.92868 1.324...","POLYGON ((103.93077 1.32784, 103.93257 1.32796...",0.000000,57.458946,20.847763,0.000000,0.000000,0.000000,TRUNK


#### Combining both Planning area geojson data with population planning area data from department of Statistics

In [25]:
df_filtered_population = df_population_planning_area[df_population_planning_area['Type of Dwelling'] == 'Total'] ## filter for total numbers for type of dwelling
df_filtered_population = df_filtered_population[df_filtered_population['Age Group'] =='Total'] ## adding everyone in the age group
df_filtered_population = df_filtered_population[df_filtered_population['Subzone'] =='Total'] ## adding everyone in the subzone
df_filtered_population['Planning Area'] = df_filtered_population['Planning Area'].str.upper() ## printing for only filtered one

df_planning_area_population = df_filtered_population.merge(df_planning_area, left_on='Planning Area', right_on='pln_area_n', how='left') ## joining the 2 dataframes to see if any discrepancies

# Rename the column '2024' to 'population_size'
df_planning_area_population.rename(columns={'2024': 'population_size'}, inplace=True)

#replace - with 0 for population size column
df_planning_area_population['population_size'] = df_planning_area_population['population_size'].replace('-', 0) 

# Display the updated dataframe to verify the change
df_planning_area_population

## eliminating upper and lower casae issue 
planning_area_location['pln_area_n'] = planning_area_location['pln_area_n'].str.lower()
df_planning_area_population['pln_area_n'] = df_planning_area_population['pln_area_n'].str.lower()

# Merge the two dataframes on the 'pln_area_n' column
combined_planning_area = pd.merge(df_planning_area_population, planning_area_location, on='pln_area_n', how='left')
combined_planning_area


,Planning Area,Subzone,Age Group,Type of Dwelling,population_size,id,pln_area_n,geojson
0,TOTAL,Total,Total,Total,4180870,NaN,NaN,NaN
1,ANG MO KIO,Total,Total,Total,159340,113.0,ang mo kio,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.8..."
2,BEDOK,Total,Total,Total,276840,114.0,bedok,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.9..."
3,BISHAN,Total,Total,Total,87930,115.0,bishan,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.8..."
4,BOON LAY,Total,Total,Total,20,116.0,boon lay,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.7..."
5,BUKIT BATOK,Total,Total,Total,167750,117.0,bukit batok,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.7..."
6,BUKIT MERAH,Total,Total,Total,148270,118.0,bukit merah,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.8..."
7,BUKIT PANJANG,Total,Total,Total,138050,119.0,bukit panjang,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.7..."
8,BUKIT TIMAH,Total,Total,Total,83570,120.0,bukit timah,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.7..."
9,CENTRAL WATER CATCHMENT,Total,Total,Total,0,121.0,central water catchment,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.8..."


#### Determining the list of planning areas an original non buffered bus route passes through

In [26]:
# Ensure 'geojson' column does not contain NaN or invalid values in combined_planning_area
combined_planning_area_cleaned = combined_planning_area.dropna(subset=['geojson'])

# Convert the 'geojson' field to valid geometries using shapely's 'shape' method
def parse_geojson(geojson_str):
    try:
        return shape(json.loads(geojson_str))  # Convert GeoJSON string to geometry
    except (TypeError, json.JSONDecodeError):
        return None  # Handle invalid JSON

# Apply parsing to convert geojson to geometries
combined_planning_area_cleaned['geometry'] = combined_planning_area_cleaned['geojson'].apply(parse_geojson)

# Filter out any rows with invalid geometries (None)
combined_planning_area_cleaned = combined_planning_area_cleaned.dropna(subset=['geometry'])

# Convert the cleaned DataFrame to a GeoDataFrame for spatial operations
combined_planning_area_gdf = gpd.GeoDataFrame(combined_planning_area_cleaned, geometry='geometry')

# Ensure the `categorised_bus_mrt_combined_gdf_overlap` dataframe is a GeoDataFrame
# Assuming 'original_bus_route_geom' already contains valid geometries
bus_routes_gdf = gpd.GeoDataFrame(categorised_bus_mrt_combined_gdf_overlap, geometry=categorised_bus_mrt_combined_gdf_overlap['original_bus_route_geom'])

# Ensure both GeoDataFrames use the same coordinate reference system (CRS)
bus_routes_gdf = bus_routes_gdf.set_crs(epsg=4326)  # Assuming WGS 84 (EPSG:4326)
combined_planning_area_gdf = combined_planning_area_gdf.set_crs(epsg=4326)

# Spatial join: Check which planning areas the bus routes intersect
joined_gdf = gpd.sjoin(bus_routes_gdf, combined_planning_area_gdf, how='left', predicate='intersects')

# Group the results by each bus route and collect the planning areas it passes through
grouped_planning_areas = joined_gdf.groupby('ServiceNo')['pln_area_n'].apply(lambda x: list(x.unique())).reset_index()

# Merge this new list of planning areas back into the bus routes GeoDataFrame
categorised_bus_mrt_combined_gdf_overlap_with_planning_areas = pd.merge(bus_routes_gdf, grouped_planning_areas, on='ServiceNo', how='left')

# Rename the new column for clarity (e.g., 'planning_areas')
categorised_bus_mrt_combined_gdf_overlap_with_planning_areas.rename(columns={'pln_area_n': 'planning_areas'}, inplace=True)

# Display the result
categorised_bus_mrt_combined_gdf_overlap_with_planning_areas

C:\Users\brand\AppData\Local\Temp\ipykernel_12116\2081018733.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_planning_area_cleaned['geometry'] = combined_planning_area_cleaned['geojson'].apply(parse_geojson)


,ServiceNo,original_bus_route_geom,buffered_bus_route_geom,NS_MRT_geom,EW_MRT_geom,DT_MRT_geom,CC_MRT_geom,NE_MRT_geom,TE_MRT_geom,Category,geometry,planning_areas
0,10,"LINESTRING (103.76868 1.29202, 103.76821 1.292...","POLYGON ((103.76596 1.29579, 103.76598 1.29629...",5.227974,20.959093,13.498189,30.487208,5.731992,13.614954,TRUNK,"LINESTRING (103.76868 1.29202, 103.76821 1.292...","[bukit merah, downtown core, queenstown, cleme..."
1,100,"LINESTRING (103.78715 1.31167, 103.78688 1.311...","POLYGON ((103.78204 1.31309, 103.78204 1.31311...",6.848529,34.687877,17.601992,28.508462,15.214120,3.682970,TRUNK,"LINESTRING (103.78715 1.31167, 103.78688 1.311...","[bukit merah, downtown core, rochor, queenstow..."
2,100A,"LINESTRING (103.87136 1.35037, 103.87167 1.348...","POLYGON ((103.86784 1.34646, 103.86805 1.34699...",0.000000,13.287873,15.697995,18.887485,36.216515,0.000000,TRUNK,"LINESTRING (103.87136 1.35037, 103.87167 1.348...","[geylang, toa payoh, serangoon]"
3,101,"LINESTRING (103.87136 1.35037, 103.87163 1.348...","POLYGON ((103.87278 1.35408, 103.87483 1.35604...",0.000000,0.000000,0.000000,13.940347,55.021526,0.000000,TRUNK,"LINESTRING (103.87136 1.35037, 103.87163 1.348...","[hougang, serangoon]"
4,102,"LINESTRING (103.86909 1.39288, 103.8691 1.3934...","POLYGON ((103.86533 1.40187, 103.86501 1.40198...",0.000000,0.000000,0.000000,0.000000,24.791770,0.000000,TRUNK,"LINESTRING (103.86909 1.39288, 103.8691 1.3934...","[hougang, sengkang, seletar]"
...,...,...,...,...,...,...,...,...,...,...,...,...
551,991C,"LINESTRING (103.7363 1.37665, 103.73601 1.3764...","POLYGON ((103.73984 1.37725, 103.74025 1.37699...",54.928368,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,"LINESTRING (103.7363 1.37665, 103.73601 1.3764...","[bukit batok, choa chu kang]"
552,992,"LINESTRING (103.72925 1.3575, 103.73042 1.3562...","POLYGON ((103.73244 1.35314, 103.73242 1.35313...",16.369616,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,"LINESTRING (103.72925 1.3575, 103.73042 1.3562...","[bukit batok, tengah]"
553,993,"LINESTRING (103.73549 1.34197, 103.73577 1.342...","POLYGON ((103.73249 1.34396, 103.7325 1.34397,...",13.656012,33.195281,0.000000,0.000000,0.000000,0.000000,TRUNK,"LINESTRING (103.73549 1.34197, 103.73577 1.342...","[jurong east, bukit batok, tengah]"
554,9A,"LINESTRING (103.92899 1.32472, 103.92868 1.324...","POLYGON ((103.93077 1.32784, 103.93257 1.32796...",0.000000,57.458946,20.847763,0.000000,0.000000,0.000000,TRUNK,"LINESTRING (103.92899 1.32472, 103.92868 1.324...","[bedok, tampines, pasir ris]"


#### Estimating the total population served by each bus service. This is achieved by summing the number of residents in the population areas that the bus route passes through.

In [27]:
# Ensure population_size column exists in df_planning_area_population and is numeric
df_planning_area_population['population_size'] = pd.to_numeric(df_planning_area_population['population_size'], errors='coerce').fillna(0)

# Create a dictionary to map planning area names to population sizes
planning_area_population_dict = dict(zip(df_planning_area_population['pln_area_n'], df_planning_area_population['population_size']))

# Function to calculate total population served for a row
def calculate_population_served(planning_areas):
    return sum([planning_area_population_dict.get(area.lower(), 0) for area in planning_areas])

# Apply the function to each row of mrt_bus_gdf_with_planning_areas
categorised_bus_mrt_combined_gdf_overlap_with_planning_areas['population_served'] = categorised_bus_mrt_combined_gdf_overlap_with_planning_areas['planning_areas'].apply(calculate_population_served)

# Display the updated dataframe
categorised_bus_mrt_combined_gdf_overlap_with_planning_areas

# Exporting as feature
categorised_bus_mrt_combined_gdf_overlap_with_planning_areas[['ServiceNo', 'planning_areas', 'population_served']].to_csv("data/categorised_bus_mrt_combined_gdf_overlap_with_planning_areas.csv", index=False)